In [268]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action = 'ignore')

import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression




In [269]:
#read crime data
missing_values = ['UNKNOWN', 'U']
df_crime = pd.read_csv("NYPD_Complaint_Data_Current__Year_To_Date_.csv", na_values=missing_values)

<h1>Data Inspection</h1>

In [270]:
#check data
df_crime.info()

#check target feature
df_crime['OFNS_DESC'].value_counts().head(100)

print(df_crime['PD_DESC'].value_counts())

print(df_crime['PD_CD'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449506 entries, 0 to 449505
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CMPLNT_NUM                449506 non-null  int64  
 1   ADDR_PCT_CD               449506 non-null  int64  
 2   BORO_NM                   448355 non-null  object 
 3   CMPLNT_FR_DT              449506 non-null  object 
 4   CMPLNT_FR_TM              449506 non-null  object 
 5   CMPLNT_TO_DT              409416 non-null  object 
 6   CMPLNT_TO_TM              409568 non-null  object 
 7   CRM_ATPT_CPTD_CD          449345 non-null  object 
 8   HADEVELOPT                1577 non-null    object 
 9   HOUSING_PSA               32583 non-null   float64
 10  JURISDICTION_CODE         449023 non-null  float64
 11  JURIS_DESC                449506 non-null  object 
 12  KY_CD                     449506 non-null  int64  
 13  LAW_CAT_CD                449506 non-null  o

<h1>Preprocessing</h1>

In [271]:
df_crime.head(10)

#drop uncesseary data
df_crime.drop(['CMPLNT_NUM','JURISDICTION_CODE','PARKS_NM','STATION_NAME','KY_CD', 'PATROL_BORO',
                'TRANSIT_DISTRICT','HADEVELOPT','HOUSING_PSA','CRM_ATPT_CPTD_CD', 
                'X_COORD_CD', 'Y_COORD_CD', "CMPLNT_TO_DT", "CMPLNT_TO_TM"
                ],axis='columns', inplace=True)

df_crime

,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,JURIS_DESC,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PD_CD,PD_DESC,...,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,69,NaN,12/17/2021,22:13:00,N.Y. POLICE DEPT,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,25-44,BLACK,M,25-44,BLACK,M,40.645647,-73.902876,"(40.64564719600002, -73.90287588699994)",POINT (-73.90287588699994 40.64564719600002)
1,113,NaN,12/17/2021,06:21:00,N.Y. POLICE DEPT,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,NaN,NaN,NaN,25-44,BLACK,F,40.682499,-73.787269,"(40.682499421000045, -73.78726915499993)",POINT (-73.78726915499993 40.682499421000045)
2,113,NaN,12/13/2021,20:05:00,N.Y. POLICE DEPT,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,NaN,NaN,NaN,25-44,BLACK,M,40.689218,-73.791449,"(40.689218058000044, -73.79144856999993)",POINT (-73.79144856999993 40.689218058000044)
3,52,BRONX,12/07/2021,22:49:00,N.Y. POLICE DEPT,FELONY,NaN,DANGEROUS WEAPONS,792.0,WEAPONS POSSESSION 1 & 2,...,NaN,NaN,NaN,NaN,NaN,E,40.882718,-73.881247,"(40.88271780700006, -73.88124724999993)",POINT (-73.88124724999993 40.88271780700006)
4,113,NaN,12/06/2021,17:25:00,N.Y. POLICE DEPT,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,25-44,BLACK,M,25-44,BLACK,M,40.696483,-73.776680,"(40.69648322200004, -73.77667979199998)",POINT (-73.77667979199998 40.69648322200004)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449501,60,BROOKLYN,01/08/2021,07:00:00,N.Y. POLICE DEPT,VIOLATION,FRONT OF,HARRASSMENT 2,637.0,"HARASSMENT,SUBD 1,CIVILIAN",...,NaN,NaN,NaN,45-64,WHITE,F,40.576003,-74.000891,"(40.57600335300003, -74.00089057799994)",POINT (-74.00089057799994 40.57600335300003)
449502,83,BROOKLYN,01/02/2021,04:50:00,N.Y. POLICE DEPT,FELONY,NaN,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",...,NaN,NaN,NaN,NaN,NaN,E,40.699369,-73.931555,"(40.69936940200005, -73.93155521999995)",POINT (-73.93155521999995 40.69936940200005)
449503,49,BRONX,01/02/2021,09:35:00,N.Y. POLICE DEPT,MISDEMEANOR,INSIDE,PETIT LARCENY,333.0,"LARCENY,PETIT FROM STORE-SHOPL",...,25-44,WHITE,M,NaN,NaN,D,40.854092,-73.867718,"(40.854092353000055, -73.86771750999998)",POINT (-73.86771750999998 40.854092353000055)
449504,101,QUEENS,01/06/2021,19:40:00,N.Y. POLICE DEPT,VIOLATION,INSIDE,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",...,25-44,WHITE HISPANIC,M,45-64,WHITE HISPANIC,F,40.596012,-73.742116,"(40.596011813000075, -73.74211638099996)",POINT (-73.74211638099996 40.596011813000075)


<h2>Missing data</h2>

In [272]:
#search missing data
df_crime.isnull().sum().sort_values(ascending=False)

SUSP_AGE_GROUP              255874
SUSP_RACE                   199688
SUSP_SEX                    183172
VIC_RACE                    125810
VIC_AGE_GROUP               116573
LOC_OF_OCCUR_DESC            74263
PREM_TYP_DESC                 1270
BORO_NM                       1151
PD_CD                          483
PD_DESC                        483
OFNS_DESC                        9
ADDR_PCT_CD                      0
Lat_Lon                          0
Longitude                        0
Latitude                         0
VIC_SEX                          0
RPT_DT                           0
LAW_CAT_CD                       0
JURIS_DESC                       0
CMPLNT_FR_TM                     0
CMPLNT_FR_DT                     0
New Georeferenced Column         0
dtype: int64

In [273]:
#drop rows without important information
df_crime = df_crime.dropna(subset=['OFNS_DESC', 'VIC_RACE', 'VIC_SEX', 'VIC_AGE_GROUP','SUSP_SEX', 'SUSP_AGE_GROUP', 'SUSP_RACE',
                                    'CMPLNT_FR_DT', 'LOC_OF_OCCUR_DESC',
                                    
                                    'PREM_TYP_DESC','PD_CD','PD_DESC'
                                    
                                    ], how='any')



len(df_crime)

114922

In [274]:
#fill BORO_NM
precincts = {
    "MANHATTAN" : [1, 5, 6, 7, 9, 10, 13, 14, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 30, 32, 33, 34],
    "BRONX": [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52],
    "BROOKLYN": [60, 61, 62, 63, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 81, 83, 84, 88, 90, 94],
    "QUEENS" : [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115],
    "STATEN ISLAND": [120, 121, 122, 123]
}

for i in df_crime.loc[df_crime['BORO_NM'].isna()].index:
    for key in precincts.keys():
        if df_crime.loc[i,['ADDR_PCT_CD']].item() in precincts[key]:
            df_crime.loc[i,['BORO_NM']] = key


df_crime['BORO_NM'].value_counts()


BROOKLYN         33028
BRONX            28531
QUEENS           25767
MANHATTAN        21932
STATEN ISLAND     5664
Name: BORO_NM, dtype: int64

<h3>Text Preprocessing & Grouping</h3>

In [275]:
#group crime
violent = ['MURDER & NON-NEGL. MANSLAUGHTER','HOMICIDE-NEGLIGENT-VEHICLE', 'MURDER & NON-NEGL. MANSLAUGHTE',
    'HOMICIDE-NEGLIGENT,UNCLASSIFIED','HOMICIDE-NEGLIGENT-VEHICLE','HOMICIDE-NEGLIGENT,UNCLASSIFIE', 
    'FELONY ASSAULT', 'ASSAULT 3 & RELATED OFFENSES','JOSTLING', 'RAPE','SEX CRIMES','FORCIBLE TOUCHING',
    'ROBBERY','OFFENSES RELATED TO CHILDREN','KIDNAPPING & RELATED OFFENSES','KIDNAPPING',
    "OFFENSES AGAINST THE PERSON", "ENDAN WELFARE INCOMP", "DISORDERLY CONDUCT", "CHILD ABANDONMENT/NON SUPPORT"]

traffic = ['OTHER TRAFFIC INFRACTION','VEHICLE AND TRAFFIC LAWS','INTOXICATED & IMPAIRED DRIVING',
           'INTOXICATED/IMPAIRED DRIVING','UNAUTHORIZED USE OF A VEHICLE 3 (UUV)','PARKING OFFENSES',
           'UNAUTHORIZED USE OF A VEHICLE']

non_violent = ['THEFT-FRAUD','FORGERY','FRAUDS','OFFENSES INVOLVING FRAUD','FRAUDULENT ACCOSTING','ARSON',
               'CRIMINAL TRESPASS','DANGEROUS WEAPONS','UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS',
               'UNLAWFUL POSS. WEAP. ON SCHOOL','Tresspassing','HARRASSMENT 2','OFFENSES AGAINST PUBLIC SAFETY',
               'HARASSMENT','PROSTITUTION & RELATED OFFENSES', 'CRIMINAL MISCHIEF & RELATED OF', "MISCELLANEOUS PENAL LAW",
               "OFF. AGNST PUB ORD SENSBLTY &", "OFFENSES AGAINST PUBLIC ADMINI", "ADMINISTRATIVE CODE", "ANTICIPATORY OFFENSES",
               "GAMBLING", "ALCOHOLIC BEVERAGE CONTROL LAW", "LOITERING/GAMBLING (CARDS, DIC", "ESCAPE 3"]

theft = ['GRAND LARCENY','PETIT LARCENY','OTHER OFFENSES RELATED TO THEF','THEFT OF SERVICES',
         'OTHER OFFENSES RELATED TO THEFT', 'PETIT LARCENY','GRAND LARCENY OF MOTOR VEHICLE',
         'POSSESSION OF STOLEN PROPERTY','POSSESSION OF STOLEN PROPERTY 5','BURGLARY', "PETIT LARCENY OF MOTOR VEHICLE", "BURGLAR'S TOOLS"]

drugs = ['DANGEROUS DRUGS','LOITERING FOR DRUG PURPOSES']

unclassified = ["NYS LAWS-UNCLASSIFIED FELONY", "OTHER STATE LAWS (NON PENAL LA", "AGRICULTURE & MRKTS LAW-UNCLASSIFIED", "OTHER STATE LAWS", 
                "NYS LAWS-UNCLASSIFIED VIOLATION"]



crime_group = []

for row in df_crime.OFNS_DESC:
    if (row in violent) == True:
        crime_group.append('VIOLENT')

    elif (row in traffic) == True:
        crime_group.append('TRAFFIC')

    elif (row in non_violent) == True:
        crime_group.append('NON_VIOLENT')

    elif (row in theft) == True:
        crime_group.append('THEFT')

    elif (row in drugs) == True:
        crime_group.append('DRUGS')

    elif (row in unclassified) == True:
        crime_group.append('UNCLASSIFIED')

    else :
        crime_group.append('UNCLASSIFIED')

df_crime["CM_GROUP"] = crime_group
df_crime.groupby("CM_GROUP")

df_crime["CM_GROUP"].value_counts()


NON_VIOLENT     62023
VIOLENT         44862
THEFT            7215
TRAFFIC           676
UNCLASSIFIED      106
DRUGS              40
Name: CM_GROUP, dtype: int64

In [276]:
df_crime = df_crime.replace({'STREET':'STREET','RESIDENCE - APT. HOUSE': 'RESIDENCE',
                        'PARKING LOT/GARAGE (PUBLIC)':'PUBLIC_PLACE',
                         'RESIDENCE-HOUSE' : 'RESIDENCE',
                         'DEPARTMENT STORE':'STORE',
                         'RESIDENCE - PUBLIC HOUSING':'RESIDENCE',
                         'HOMELESS SHELTER':'RESIDENCE',
                         'TRANSIT - NYC SUBWAY':'TRANSIT',
                         'HOTEL/MOTEL':'RESIDENCE',
                         'CHAIN STORE':'STORE',
                         'RESTAURANT/DINER':'RESTAURANT',
                         'GYM/FITNESS FACILITY':'STORE',
                         'SMALL MERCHANT':'STORE_CAUTION',
                         'GROCERY/BODEGA':'STORE',
                         'OTHER':'OTHER',
                         'PUBLIC BUILDING':'PUBLIC_PLACE',
                         'DRUG STORE':'STORE_CAUTION',
                         'PARK/PLAYGROUND':'PUBLIC_PLACE',
                         'CLOTHING/BOUTIQUE':'STORE',
                         'COMMERCIAL BUILDING':'PUBLIC_PLACE',
                         'HIGHWAY/PARKWAY':'VEHICLE',
                         'BANK':'PUBLIC_BUILDING_SAFTY',
                         'MAILBOX INSIDE':'OTHER',
                         'FAST FOOD':'STORE',
                         'DOCTOR/DENTIST OFFICE':'STORE',
                         'DRY CLEANER/LAUNDRY':'STORE',
                         'FOOD SUPERMARKET':'STORE',
                         'HOSPITAL':'PUBLIC_BUILDING_SAFTY',
                         'AIRPORT TERMINAL':'PUBLIC_BUILDING_SAFTY',
                         'CANDY STORE':'PUBLIC_BUILDING_SAFTY',
                          'BAR/NIGHT CLUB':'STORE_CAUTION',
                         'STORE UNCLASSIFIED':'STORE',
                          'PARKING LOT/GARAGE (PRIVATE)':'PUBLIC_PLACE',
                          'PUBLIC SCHOOL':'PUBLIC_BUILDING_SAFTY',
                           'TELECOMM. STORE':'STORE',
                          'CHECK CASHING BUSINESS':'STORE_CAUTION',
                          'BEAUTY & NAIL SALON':'STORE',
                          'BUS (NYC TRANSIT)':'VEHICLE',
                          'LIQUOR STORE':'STORE_CAUTION',
                            'CONSTRUCTION SITE':'STREET','BUS STOP':'TRANSIT','BUS (OTHER)':'VEHICLE',
                        'CHURCH':'PUBLIC_PLACE',
                         'FACTORY/WAREHOUSE':'OTHER',
                         'TAXI/LIVERY (UNLICENSED)':'VEHICLE',
                         'GAS STATION':'STORE_CAUTION',
                        'OPEN AREAS (OPEN LOTS)':'STREET',
                          'ATM':'STREET',
                         'VARIETY STORE':'STORE',
                         'SHOE':'STORE',
                         'MAILBOX OUTSIDE':'STREET',
                         'SOCIAL CLUB/POLICY':'RESTAURANT',
                         'BOOK/CARD':'STORE',
                        'BUS TERMINAL':'TRANSIT',
                        'CEMETERY':'STREET',
                         'TUNNEL':'STREET',
                         'MARINA/PIER':'STREET',
                         'OTHER HOUSE OF WORSHIP':'PUBLIC_PLACE',
                         'BRIDGE':'STREET',
                         'TAXI (YELLOW LICENSED)':'VEHICLE',
                         'SYNAGOGUE':'PUBLIC_PLACE',
                         'MOSQUE':'PUBLIC_PLACE',
                         'FERRY/FERRY TERMINAL':'TRANSIT',
                         'LOAN COMPANY':'STORE_CAUTION',
                         'DAYCARE FACILITY':'PUBLIC_BUILDING_SAFTY',
                         'PHOTO/COPY':'STORE',
                         'ABANDONED BUILDING':'STREET',
                         'TRAMWAY':'TRANSIT',
                         'JEWELRY':'STORE_CAUTION',
                         'VIDEO STORE':'STORE',
                        'TRANSIT FACILITY (OTHER)':'TRANSIT',
                        'STORAGE FACILITY':'OTHER',
                        'TAXI (LIVERY LICENSED)':'TRANSIT',
                        'PRIVATE/PAROCHIAL SCHOOL':'PUBLIC_BUILDING_SAFTY'})
df_crime.PREM_TYP_DESC.unique()

df_crime.PREM_TYP_DESC.value_counts()
df_crime.VIC_RACE.value_counts()

BLACK                             48366
WHITE HISPANIC                    30360
WHITE                             18500
ASIAN / PACIFIC ISLANDER          10352
BLACK HISPANIC                     6940
AMERICAN INDIAN/ALASKAN NATIVE      404
Name: VIC_RACE, dtype: int64

In [277]:
#convert datetime(String) to datetime(datetime)
date_format = "%m/%d/%Y"
datetime = []
try:
    datetime.datetime.strptime(df_crime['CMPLT_FR_DT'], date_format))
except ValueError:
    print("Error")


AttributeError: 'list' object has no attribute 'datetime'

<h3>Data normalization</h3>

<h4>Categorical data encoding</h4>

In [297]:
df_ordinal = df_crime.copy()
df_oneHot = df_crime.copy()
df_label = df_crime.copy()

ord_enc = OrdinalEncoder()
label_enc = LabelEncoder()
onHot_enc = OneHotEncoder(sparse=False)

df_crime_categorical = df_crime.select_dtypes(include=["object"])

df_crime_numeric = df_crime.select_dtypes(include=["int64", "float64"])

print(df_crime_categorical.columns)
print(df_crime_numeric.columns)

#Conver categorical columns with ordinal encoder
df_ordinal[df_crime_categorical.columns] = df_crime_categorical
ord_enc.fit(df_crime_categorical)

df_ordinal = df_ordinal.reset_index() 
df_ordinal[df_crime_categorical.columns]= pd.DataFrame(ord_enc.transform(df_crime_categorical))
df_ordinal[df_crime_categorical.columns]

print(df_ordinal.head(10))


#Conver categorical columns with label encoder
df_label = df_label.reset_index()

for column in df_crime_categorical.columns:
    label_enc.fit(df_crime_categorical[column])
    df_label[column] = pd.DataFrame(label_enc.transform(df_crime_categorical[column]))

print(df_label.head(10))

# #Convert categorical columnw with OneHowEncoder
df_oneHot = df_oneHot.reset_index()

for column in df_crime_categorical.columns:
    if len(df_oneHot[column].unique()) == 2:
        df_oneHot[column] = pd.get_dummies(df_oneHot[column], drop_first=True)

df_oneHot = pd.get_dummies(df_oneHot)

print(df_oneHot.head(10))
    



Index(['BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'JURIS_DESC', 'LAW_CAT_CD',
       'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT',
       'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE',
       'VIC_SEX', 'Lat_Lon', 'New Georeferenced Column', 'CM_GROUP'],
      dtype='object')
Index(['ADDR_PCT_CD', 'PD_CD', 'Latitude', 'Longitude'], dtype='object')
   index  ADDR_PCT_CD  BORO_NM  CMPLNT_FR_DT  CMPLNT_FR_TM  JURIS_DESC  \
0     71           44      0.0        1193.0         260.0         5.0   
1     75           46      0.0        1183.0         798.0         5.0   
2     76          105      3.0        1179.0        1140.0         5.0   
3     77           48      0.0        1176.0        1020.0         5.0   
4     82           75      1.0        1188.0         580.0         5.0   
5     83           47      0.0        1173.0         430.0         4.0   
6     89           61      1.0        1188.0         180.0         5.0   
7     9

<h4>Normalization</h4>

In [ ]:
#target feature = CM GROUP
y_ord = df_ordinal['CM_GROUP'].copy()
y_label = df_label['CM_GROUP'].copy()
y_onHot = df_label['CM_GROUP'].copy()

X_ord = df_ordinal.drop(['CM_GROUP'], axis=1)
X_label = df_label.drop(['CM_GROUP'], axis=1)
X_onHot = df_oneHot.drop(['CM_GROUP'], axis=1)

X_ord_train, X_ord_test, y_ord_train, y_ord_test = train_test_split(X_ord, y_ord, random_state=0)
X_label_train, X_label_test, y_label_train, y_label_test = train_test_split(X_label, y_label, random_state=0)
X_onHot_train, X_onHot_test, y_onHot_train, y_onHot_test = train_test_split(X_onHot, y_onHot random_state=0)
